# Credit: [Rongyao Fang](https://github.com/rongyaofang)

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.checkpoint import checkpoint
device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
from sklearn.metrics import roc_auc_score
# from tensorboardX import SummaryWriter
import tensorboard_logger as tb_logger

In [2]:
PARAMS = {
    "input_channels": 1,
    "output_channels": 2,
    "input_transform_fn": lambda x: x / 128. - 1.,
    "input_conv_channels": 32,
    'down_structure': [2,2,2],
#     'down_structure': [3, 8, 4],
    'activation_fn': lambda: nn.LeakyReLU(0.1, inplace=True),
    # "normalization_fn": lambda c: nn.GroupNorm(num_groups=4, num_channels=c),
    "normalization_fn": lambda c: nn.BatchNorm3d(num_features=c),
    "drop_rate": 0,
    'growth_rate': 32,
    'bottleneck': 4,
    'compression': 2,
    'use_memonger': True  # ~2.2x memory efficiency (batch_size), 25%~30% slower on GTX 1080.
}

In [3]:
def densenet3d(with_segment=False, snapshot=None, **kwargs):
    for k, v in kwargs.items():
        assert k in PARAMS
        PARAMS[k] = v
    print("Model hyper-parameters:", PARAMS)
    if with_segment:
        model = DenseSharp()
        print("Using DenseSharp model.")
    else:
        model = DenseNet()
        print("Using DenseNet model.")
    print(model)
    if snapshot is None:
        initialize(model.modules())
        print("Random initialized.")
    else:
        state_dict = torch.load(snapshot)
        model.load_state_dict(state_dict)
        print("Load weights from `%s`," % snapshot)
    return model

In [4]:
def initialize(modules):
    for m in modules:
        if isinstance(m, nn.Conv3d) or isinstance(m, nn.ConvTranspose3d):
            nn.init.kaiming_uniform_(m.weight, mode='fan_in')
        elif isinstance(m, nn.Linear):
            nn.init.kaiming_uniform_(m.weight, mode='fan_in')
            m.bias.data.zero_()

In [5]:
class ConvBlock(nn.Sequential):
    def __init__(self, in_channels):
        super(ConvBlock, self).__init__()

        growth_rate = PARAMS['growth_rate']
        bottleneck = PARAMS['bottleneck']
        activation_fn = PARAMS['activation_fn']
        normalization_fn = PARAMS['normalization_fn']

        self.in_channels = in_channels
        self.growth_rate = growth_rate
        self.use_memonger = PARAMS['use_memonger']
        self.drop_rate = PARAMS['drop_rate']

        # TODO: consider bias term in conv with GN
        self.add_module('norm_1', normalization_fn(in_channels))
        self.add_module('act_1', activation_fn())
        self.add_module('conv_1', nn.Conv3d(in_channels, bottleneck * growth_rate, kernel_size=1, stride=1,
                                            padding=0, bias=True))

        self.add_module('norm_2', normalization_fn(bottleneck * growth_rate))
        self.add_module('act_2', activation_fn())
        self.add_module('conv_2', nn.Conv3d(bottleneck * growth_rate, growth_rate, kernel_size=3, stride=1,
                                            padding=1, bias=True))

    def forward(self, x):
        super_forward = super(ConvBlock, self).forward
        if self.use_memonger:
            new_features = checkpoint(super_forward, x)
        else:
            new_features = super_forward(x)
        if self.drop_rate > 0:
            new_features = F.dropout(new_features, p=self.drop_rate, training=self.training)
        return torch.cat([x, new_features], 1)

    @property
    def out_channels(self):
        return self.in_channels + self.growth_rate

In [6]:
class TransmitBlock(nn.Sequential):
    def __init__(self, in_channels, is_last_block):
        super(TransmitBlock, self).__init__()

        activation_fn = PARAMS['activation_fn']
        normalization_fn = PARAMS['normalization_fn']
        compression = PARAMS['compression']

        # print("in_channels: %s, compression: %s" % (in_channels, compression))
        assert in_channels % compression == 0

        self.in_channels = in_channels
        self.compression = compression

        self.add_module('norm', normalization_fn(in_channels))
        self.add_module('act', activation_fn())
        if not is_last_block:
            self.add_module('conv', nn.Conv3d(in_channels, in_channels // compression, kernel_size=(1, 1, 1),
                                              stride=1, padding=0, bias=True))
            self.add_module('pool', nn.AvgPool3d(kernel_size=2, stride=2, padding=0))
        else:
            self.compression = 1

    @property
    def out_channels(self):
        return self.in_channels // self.compression

In [7]:
class Lambda(nn.Module):
    def __init__(self, lambda_fn):
        super(Lambda, self).__init__()
        self.lambda_fn = lambda_fn

    def forward(self, x):
        return self.lambda_fn(x)

In [8]:
class DenseNet(nn.Module):

    def __init__(self):

        super(DenseNet, self).__init__()

        input_channels = PARAMS['input_channels']
        input_transform_fn = PARAMS['input_transform_fn']
        input_conv_channels = PARAMS['input_conv_channels']
        normalization_fn = PARAMS['normalization_fn']
        activation_fn = PARAMS['activation_fn']
        down_structure = PARAMS['down_structure']
        output_channels = PARAMS['output_channels']

        self.features = nn.Sequential()
        if input_transform_fn is not None:
            self.features.add_module("input_transform", Lambda(input_transform_fn))
        self.features.add_module("init_conv", nn.Conv3d(input_channels, input_conv_channels, kernel_size=3,
                                                        stride=1, padding=1, bias=True))
        self.features.add_module("init_norm", normalization_fn(input_conv_channels))
        self.features.add_module("init_act", activation_fn())

        channels = input_conv_channels
        for i, num_layers in enumerate(down_structure):
            for j in range(num_layers):
                conv_layer = ConvBlock(channels)
                self.features.add_module('denseblock{}_layer{}'.format(i + 1, j + 1), conv_layer)
                channels = conv_layer.out_channels
                # print(i, j, channels)

            trans_layer = TransmitBlock(channels, is_last_block=i == len(down_structure) - 1)
            self.features.add_module('transition%d' % (i + 1), trans_layer)
            channels = trans_layer.out_channels

        self.classifier = nn.Linear(channels, output_channels)

    def forward(self, x, **return_opts):
        # o = x
        # for i, layer in enumerate(self.features):
        #     o = layer(o)
        #     print(i, layer, o.shape)

        batch_size, _, d, h, w = x.size()

        features = self.features(x)
        pooled = F.adaptive_avg_pool3d(features, 1).view(batch_size, -1)
        scores = self.classifier(pooled)

        if len(return_opts) == 0:
            return scores

        # return other features in one forward
        for opt in return_opts:
            assert opt in {"return_features", "return_cam"}
        # print(return_opts)

        ret = dict(scores=scores)

        if 'return_features' in return_opts and return_opts['return_features']:
            ret['features'] = features

        if 'return_cam' in return_opts and return_opts['return_cam']:
            weight = self.classifier.weight.unsqueeze(-1).unsqueeze(-1).unsqueeze(-1)
            bias = self.classifier.bias
            cam_raw = F.conv3d(features, weight, bias)
            cam = F.interpolate(cam_raw, size=(d, h, w), mode='trilinear', align_corners=True)
            ret['cam'] = F.softmax(cam, dim=1)
            ret['cam_raw'] = F.softmax(cam_raw, dim=1)
        return ret

In [9]:
from torch.utils.data import Dataset
import random
import torch
import os
import torch.nn as nn
from tqdm import tqdm

import pandas as pd

import numpy as np

class ClfDataset(Dataset):

    def __init__(self, train=True):
        self.train = train
        data_dir = './dataset/'
        # choose the dataset
        patients_train = os.listdir(data_dir+'train_val/')
        patients_train.sort()
        patients_test = os.listdir(data_dir+'test/')
        patients_test.sort()
        labels_df = pd.read_csv('./dataset/info.csv',index_col=0)

        self.data_train = []
        self.data_test = []
        self.label = []

        for num, patient in enumerate(patients_train):
            patient_name = patient[0:-4]
            label = labels_df.get_value(patient_name, 'lable')
            path = data_dir + 'train_val/' + patient
            img_data = np.load(path)
            voxel = img_data['voxel'].astype(np.int32)
            self.data_train.append(voxel)
            self.label.append(label)

        for num, patient in enumerate(patients_test):
            path = data_dir + 'test/' + patient
            img_data = np.load(path)
            voxel = img_data['voxel'].astype(np.int32)
            self.data_test.append(voxel)
    
    def __getitem__(self, item):
        if self.train:
            patient_data = self.data_train[item]
            patient_label = self.label[item]
            return patient_data, patient_label
        else:
            patient_data = self.data_test[item]
            return patient_data
        
    def __len__(self):
        if self.train:
            return len(self.label)
        else:
            return len(self.data_test)

In [10]:
# '''[begin] load the train and validation dataset:'''
# import numpy as np
# # use np.load to import data, devide dataset into 2 parts: train_data & validation_data:
# # credit: cheez & Matthew Kerian
# # link: https://stackoverflow.com/questions/55890813/how-to-fix-object-arrays-cannot-be-loaded-when-allow-pickle-false-for-imdb-loa/56243777
# '''use older to successfully load the data:'''
# # save np.load
# np_load_old = np.load

# # modify the default parameters of np.load
# np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

# data_ineed = np.load('muchdata.npy')

# # restore np.load for future normal usage
# np.load = np_load_old

# # If you are working with the basic sample data, use maybe 2 instead of 100 here... you don't have enough data to really do this
# train_data = data_ineed[:-60]
# validation_data = data_ineed[-60:]
# '''[end] load the train and validation dataset:'''

model = densenet3d(with_segment=False, use_memonger=True).cuda()
# model = nn.DataParallel(model, device_ids=[0, 1])
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

criterion = criterion.cuda()

from torch.utils.data import DataLoader

data_train = ClfDataset(train=True)
train_data_loader = DataLoader(dataset=data_train, batch_size=32, shuffle=False)
dev_data_loader = DataLoader(dataset=data_train, batch_size=32, shuffle=False)
data_test = ClfDataset(train=False)
test_data_loader = DataLoader(dataset=data_test, batch_size=32, shuffle=False)

Model hyper-parameters: {'input_channels': 1, 'output_channels': 2, 'input_transform_fn': <function <lambda> at 0x7f6c93d2d620>, 'input_conv_channels': 32, 'down_structure': [2, 2, 2], 'activation_fn': <function <lambda> at 0x7f6c93d2d6a8>, 'normalization_fn': <function <lambda> at 0x7f6c93d2d730>, 'drop_rate': 0, 'growth_rate': 32, 'bottleneck': 4, 'compression': 2, 'use_memonger': True}
Using DenseNet model.
DenseNet(
  (features): Sequential(
    (input_transform): Lambda()
    (init_conv): Conv3d(1, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (init_norm): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (init_act): LeakyReLU(negative_slope=0.1, inplace=True)
    (denseblock1_layer1): ConvBlock(
      (norm_1): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_1): LeakyReLU(negative_slope=0.1, inplace=True)
      (conv_1): Conv3d(32, 128, kernel_size=(1, 1, 1), stride=(1, 1, 1))
     

/home/vivi/anaconda3/envs/3dunet/lib/python3.7/site-packages/ipykernel_launcher.py:30: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


In [11]:
# for epoch in range(4):  # loop over the dataset multiple times
#     running_loss = 0.0
#     for inputs, labels in train_data_loader:
#         # get the inputs; data is a list of [inputs, labels]
#         # inputs: tensor(100*(100*100)), label
#         inputs_numpy, labels = data
#         inputs = torch.from_numpy(inputs_numpy)
#         inputs = inputs.unsqueeze(0)
#         inputs = inputs.unsqueeze(0)
#         inputs = inputs.cuda()
# #         inputs = inputs.float()
#         labels = torch.from_numpy(np.asarray(labels)).float()
#         labels = labels.long() #credit: ptrblck; link: https://discuss.pytorch.org/t/runtimeerror-expected-object-of-scalar-type-long-but-got-scalar-type-float-when-using-crossentropyloss/30542/4
#         labels = labels.unsqueeze(0)

#         labels = labels.cuda()
#         # zero the parameter gradients
#         optimizer.zero_grad()

#         # forward + backward + optimize
#         outputs = model(inputs)
        
#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()

#         # print statistics
#         running_loss += loss.item()
#         if i % 20 == 19:    # print every 2000 mini-batches
#             print('[%d, %5d] loss: %.3f' %
#                   (epoch + 1, i + 1, running_loss / 20))
#             running_loss = 0.0

# print('Finished Training')

### Only train:

In [27]:
for epoch in range(100):  # loop over the dataset multiple times
    
    model.train()
    running_loss = 0.0
    for index, (inputs, labels) in enumerate(tqdm(train_data_loader)):
        # zero the parameter gradients
        optimizer.zero_grad()
        
        inputs, labels = inputs.cuda(), labels.cuda()
        
        # forward + backward + optimize
        inputs = inputs.unsqueeze(dim=1).float()
        
        inputs = F.interpolate(inputs, size=[32,32,32],mode='trilinear',align_corners=False)
        
        outputs = model(inputs)
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        # print statistics
        running_loss += loss.item()
        
        if index==11:
            break
        

    print('[%d] loss: %.5f' %
        (epoch + 1, running_loss / 352))
    running_loss = 0.0

  0%|          | 0/15 [00:00<?, ?it/s]

[1] loss: 0.01804


  0%|          | 0/15 [00:00<?, ?it/s]

[2] loss: 0.01768


  0%|          | 0/15 [00:00<?, ?it/s]

...

[98] loss: 0.00002


  0%|          | 0/15 [00:00<?, ?it/s]

[99] loss: 0.00002


 73%|███████▎  | 11/15 [00:06<00:02,  1.59it/s]

[100] loss: 0.00002


### Only Valiation:

In [12]:
PATH = './my_23_1.pth'
model = densenet3d(with_segment=False, use_memonger=True).cuda()
model.load_state_dict(torch.load(PATH))

model.eval()
dev_loss = 0
predict_value = torch.zeros(1)
true_value = torch.zeros(1)

for index, (inputs, labels) in enumerate(tqdm(train_data_loader)):
    if index<=11:
        continue

    inputs, labels = inputs.cuda(), labels.cuda()
        
        # forward + backward + optimize
    inputs = inputs.unsqueeze(dim=1).float()
        
    inputs = F.interpolate(inputs, size=[32,32,32],mode='trilinear',align_corners=False)
        
    outputs = model(inputs)
    
    test_value = F.softmax(outputs)
    test_value_1 = test_value[:,1]
    test_value_1 = test_value_1.cpu()
    predict_value = torch.cat((predict_value,test_value_1))
    
    test_value_2 = labels
    test_value_2 = test_value_2.cpu().float()
    true_value = torch.cat((true_value, test_value_2))
#     loss = criterion(outputs, labels)    
#     dev_loss+=loss.item()
    
# dev_loss = dev_loss/64
# print("%.4f" %dev_loss)

predict_value = predict_value.detach().numpy()
true_value = true_value.numpy()

from sklearn.metrics import roc_auc_score
roc_auc_score(true_value, predict_value)

 20%|██        | 3/15 [00:00<00:00, 26.57it/s]

Model hyper-parameters: {'input_channels': 1, 'output_channels': 2, 'input_transform_fn': <function <lambda> at 0x7f72c8de7ae8>, 'input_conv_channels': 32, 'down_structure': [2, 2, 2], 'activation_fn': <function <lambda> at 0x7f72c8de7b70>, 'normalization_fn': <function <lambda> at 0x7f72c8de7bf8>, 'drop_rate': 0, 'growth_rate': 32, 'bottleneck': 4, 'compression': 2, 'use_memonger': True}
Using DenseNet model.
DenseNet(
  (features): Sequential(
    (input_transform): Lambda()
    (init_conv): Conv3d(1, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (init_norm): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (init_act): LeakyReLU(negative_slope=0.1, inplace=True)
    (denseblock1_layer1): ConvBlock(
      (norm_1): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_1): LeakyReLU(negative_slope=0.1, inplace=True)
      (conv_1): Conv3d(32, 128, kernel_size=(1, 1, 1), stride=(1, 1, 1))
     

100%|██████████| 15/15 [00:01<00:00, 13.18it/s]


0.6159420289855072

### Train & Valiation:

In [11]:
date = 23 # day
number = 2 # No.
file_name = 'train_{}_{}'.format(date,number)
# writer = SummaryWriter('./runs/{}'.format(file_name))
logger = tb_logger.Logger(logdir = './runs/train_{}_{}'.format(date,number), flush_secs=2)
# val_logger = tb_logger.Logger(logdir = './runs/train_{}_{}/test'.format(date,number), flush_secs=2)

for epoch in range(200):  # loop over the dataset multiple times
    
    # for train:
    running_loss = 0.0
    # for valiation:
    dev_loss = 0
    predict_value = torch.zeros(1) # valiation predict:
    true_value = torch.zeros(1) # true lable of valiation part:
    
    for index, (inputs, labels) in enumerate(tqdm(train_data_loader)):
        
        if index<=10:# train part: 
            
            model.train()
            
            # zero the parameter gradients
            optimizer.zero_grad()
        
            inputs, labels = inputs.cuda(), labels.cuda()
        
            # forward + backward + optimize
            inputs = inputs.unsqueeze(dim=1).float()
        
            inputs = F.interpolate(inputs, size=[32,32,32],mode='trilinear',align_corners=False)
        
            outputs = model(inputs)
        
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        
            # print statistics
            running_loss += loss.item()
            
        else: # valiation part:
            
            model.eval()
            
            inputs, labels = inputs.cuda(), labels.cuda()
        
            # forward + backward + optimize
            inputs = inputs.unsqueeze(dim=1).float()
        
            inputs = F.interpolate(inputs, size=[32,32,32],mode='trilinear',align_corners=False)
        
            outputs = model(inputs)
        
            test_value = F.softmax(outputs)
            test_value_1 = test_value[:,1]
            test_value_1 = test_value_1.cpu()
            predict_value = torch.cat((predict_value,test_value_1))
    
            test_value_2 = labels
            test_value_2 = test_value_2.cpu().float()
            true_value = torch.cat((true_value, test_value_2))
            
    print('[%d] loss: %.5f' %(epoch + 1, running_loss / 352))
#     writer.add_scalar('Loss', running_loss, epoch)
    logger.log_value('loss',running_loss,epoch)
    running_loss = 0.0
        
    predict_value = predict_value.detach().numpy()
    true_value = true_value.numpy()
    auc_score = roc_auc_score(true_value, predict_value)
    print(auc_score)
#     writer.add_scalar('Auc', auc_score, epoch)
    logger.log_value('AUC',auc_score, epoch)
    
# writer.close()

  0%|          | 0/15 [00:00<?, ?it/s]

[1] loss: 0.02342
0.5475


  0%|          | 0/15 [00:00<?, ?it/s]

[2] loss: 0.02207
0.6128125


  0%|          | 0/15 [00:00<?, ?it/s]

...

[198] loss: 0.00001
0.6153125


  0%|          | 0/15 [00:00<?, ?it/s]

[199] loss: 0.00001
0.6153125


100%|██████████| 15/15 [00:07<00:00,  2.02it/s]

[200] loss: 0.00001
0.615625


In [13]:
PATH = './my_{}_{}.pth'.format(date,number)
torch.save(model.state_dict(), PATH)

In [13]:
# ./my_23_1.pth 100epochs

In [14]:
# PATH = './my_{}_{}.pth'.format(date,number)
# model = densenet3d(with_segment=False, use_memonger=True).cuda()
# model.load_state_dict(torch.load(PATH))

model.eval()
# dev_loss = 0
predict_value = torch.zeros(1)
# true_value = torch.zeros(1)

for _, inputs in enumerate(tqdm(test_data_loader)):
    
    inputs = inputs.cuda()
        # forward + backward + optimize
    inputs = inputs.unsqueeze(dim=1).float()
        
    inputs = F.interpolate(inputs, size=[32,32,32],mode='trilinear',align_corners=False)
        
    outputs = model(inputs)
    
    test_value = F.softmax(outputs)
    test_value_1 = test_value[:,1]
    test_value_1 = test_value_1.cpu()
    print(test_value_1)
    
    predict_value = torch.cat((predict_value,test_value_1))
    
#     test_value_2 = labels
#     test_value_2 = test_value_2.cpu().float()
#     true_value = torch.cat((true_value, test_value_2))


test_predict_value = predict_value.detach().numpy()


 50%|█████     | 2/4 [00:00<00:00,  4.61it/s]

tensor([9.6719e-01, 2.4004e-01, 5.4296e-01, 1.0180e-03, 6.6936e-01, 9.8357e-01,
        7.7140e-01, 4.4515e-01, 1.2292e-03, 9.8532e-01, 6.4691e-05, 9.9608e-01,
        9.9997e-01, 9.9925e-01, 5.4478e-02, 3.0596e-01, 2.2353e-01, 1.3752e-02,
        9.8629e-01, 9.9273e-01, 9.9948e-01, 9.9996e-01, 9.9574e-01, 9.9967e-01,
        1.4972e-05, 7.2746e-01, 1.0000e+00, 3.8546e-03, 8.7125e-01, 7.0966e-01,
        1.5614e-01, 1.4445e-02], grad_fn=<CopyBackwards>)
tensor([3.6944e-01, 7.3634e-01, 1.0701e-04, 9.8539e-01, 9.9012e-01, 7.7569e-01,
        9.8868e-01, 3.8570e-03, 9.9750e-01, 8.9989e-01, 9.9578e-01, 8.9807e-01,
        9.9627e-01, 5.4979e-01, 4.5971e-03, 5.5637e-01, 2.0675e-02, 4.2529e-03,
        1.1961e-02, 1.0000e+00, 4.0961e-01, 8.6671e-02, 9.6109e-01, 5.2666e-01,
        9.8714e-01, 9.9827e-01, 8.4265e-04, 5.8670e-01, 4.4646e-01, 1.5802e-01,
        9.8892e-01, 9.5951e-01], grad_fn=<CopyBackwards>)


100%|██████████| 4/4 [00:00<00:00,  5.44it/s]

tensor([4.9257e-02, 5.2016e-03, 9.5003e-01, 9.5910e-01, 6.1103e-01, 8.2570e-01,
        5.1945e-04, 6.3231e-01, 9.8702e-01, 2.2981e-01, 4.8577e-02, 9.9877e-01,
        9.9992e-01, 1.1754e-05, 9.1244e-01, 4.2973e-02, 9.6494e-01, 9.9942e-01,
        9.9805e-01, 9.8803e-01, 9.3331e-01, 9.8368e-01, 9.9721e-01, 9.9999e-01,
        7.6689e-02, 9.8950e-01, 9.2978e-01, 9.6023e-01, 9.2444e-01, 4.2890e-03,
        1.2466e-01, 1.8652e-05], grad_fn=<CopyBackwards>)
tensor([9.9974e-01, 9.0084e-01, 2.8142e-01, 9.6215e-01, 4.1416e-03, 5.2040e-04,
        8.2561e-01, 3.2699e-05, 9.9984e-01, 9.9991e-01, 9.8826e-01, 9.9996e-01,
        9.3766e-01, 8.0073e-02, 9.7807e-01, 6.1795e-07, 1.2711e-07, 1.5197e-02,
        4.6077e-01, 1.1412e-07, 9.9958e-01], grad_fn=<CopyBackwards>)


In [22]:
test_predict_value

array([0.        , 0.7681988 , 0.21929917, 0.25306442, 0.13273145,
       0.4268722 , 0.74537003, 0.3375637 , 0.69582593, 0.09741792,
       0.43878582, 0.16675016, 0.61610097, 0.44171885, 0.4440905 ,
       0.13823506, 0.35815305, 0.5310992 , 0.15788056, 0.42568162,
       0.6047288 , 0.35053736, 0.70405084, 0.5185141 , 0.8323816 ,
       0.03381161, 0.3956179 , 0.37155223, 0.03907281, 0.53791827,
       0.3630199 , 0.47596002, 0.26511502, 0.47440252, 0.30700487,
       0.03689992, 0.42741382, 0.31713384, 0.27939385, 0.640623  ,
       0.4235127 , 0.4246907 , 0.62693954, 0.52634645, 0.49056748,
       0.7664389 , 0.4124694 , 0.196875  , 0.37485656, 0.08014497,
       0.12605228, 0.16157927, 0.87304276, 0.7250747 , 0.10267787,
       0.46728963, 0.3557893 , 0.6340106 , 0.76406246, 0.5415598 ,
       0.5741628 , 0.26083955, 0.39410833, 0.79902196, 0.94410497,
       0.39074177, 0.506341  , 0.90756494, 0.6660692 , 0.14984173,
       0.42179722, 0.49830067, 0.6371415 , 0.39978033, 0.63240

In [19]:
predict_value.size

118

In [22]:
ineed.size

97

In [24]:
np.savetxt('new_23_2.csv', test_predict_value, delimiter = ',')